<a href="https://colab.research.google.com/github/GiulioAppetito/Progetto_MOBD_2122/blob/main/test_routine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
%cd /content/drive/My\ Drive/ProgettoMOBD

/content/drive/My Drive/ProgettoMOBD


In [44]:
import sklearn
import pandas as pd
import numpy as np

import dill
import cloudpickle

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

from joblib import load

np.random.seed = 123457

# FILES PER ESEGUIRE LA TEST ROUTINE
training_set_filename = "train.csv"
test_set_filename = "test_set.csv" 
preprocessor_filename = 'preprocessor.pkl'
classifier_filename = 'classifier.joblib'

In [44]:
# LETTURA TEST SET
test = pd.read_csv(test_set_filename, sep=",")
print(test.shape)

#PREPROCESSING TEST SET
x_test = test.iloc[:, :-1].to_numpy()
y_test = test.iloc[:, -1].to_numpy()

print(x_test.shape)

with open(preprocessor_filename, 'rb') as in_file:
    preprocessor = dill.load(in_file)

x_test = preprocessor.transform(x_test)
print(x_test.shape)

#CLASSIFICATORE
classifier = load(classifier_filename)
y_ts_pred = classifier.predict(x_test)

print("Accuracy on test set {}\n".format(accuracy_score(y_test, y_ts_pred)))
print("F1 score on test set {}\n".format(f1_score(y_test, y_ts_pred)))
print(confusion_matrix(y_test, y_ts_pred))